In [88]:
 import pandas as pd

In [89]:
 import sys
 sys.path.append('/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/')

In [90]:
df=pd.read_csv('/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/master_data_FINISHED - master_data.csv')
df.head() #grab the text column 

,file_name,year,Text,formality_rating,gpt_coding_final,angela_formality_rating,lyle_formality_rating,ben_formality_rating,steven_formality_rating,average_rating,interrater_reliability,overall_inter_rater_reliability,GPT3_5_NO_example,no_example_cor,GPT3_5_with_example,example_cor,GPT3_5_2_example
0,627327-miss-wimples-hoop.html,1858,"But Miss Wimple’s proud and honorable fund, fo...",NaN,fiction,1,1,1,1,1.00,1.000000,0.280369,1,0.60026,1,0.722049,1
1,628073-saccharissa-mellasys.html,1861,"The late Mr. A. Bratley, my grandfather, was i...",NaN,fiction,1,1,1,1,1.00,1.000000,0.280369,1,NaN,1,NaN,1
2,630890-wilhelmina.html,1875,"She was the gardener’s daughter, a member of t...",NaN,fiction,1,1,1,1,1.00,1.000000,0.280369,1,NaN,1,NaN,1
3,632295-a-sequel-to-mr-washington-adams-in-a-le...,1884,"I saw a little knoll near by, which would give...",NaN,fiction,1,1,0,0,0.50,-0.222222,0.280369,1,NaN,1,NaN,1
4,635408-witchcraft.html,1895,The theological student was going out to take ...,NaN,fiction,1,1,1,0,0.75,-0.222222,0.280369,1,NaN,1,NaN,1


In [91]:
#examples 
examples=pd.read_csv('/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/Examples - master_data.csv', index_col=0)
examples.head()

,year,Example,Label
file_name,,,
627327-miss-wimples-hoop.html,1858,"But Miss Wimple’s proud and honorable fund, fo...",1
628073-saccharissa-mellasys.html,1861,"The late Mr. A. Bratley, my grandfather, was i...",1
670391-died-and-gone-to-vegas.html,1995,"Nick, the young oiler, laced his fingers behin...",0
645979-the-third-window.html,1920,Yes. But she is so much a part of it that it w...,0


In [92]:
inputs = df.Text.values #save the text
print(inputs)#print some inputs to view 

['But Miss Wimple’s proud and honorable fund, for the relief of the shop, by no means fell off. As she had anticipated, her expert, and nimble needle was in steady demand by all the folks of Hendrik who had fine sewing to give out. Her earnings from this source were considerable; and, severely stinting herself in the very necessaries of life by a strained ingenuity of economy, to which the skimped delaine —turned and altered to the utter exhaustion of the cleverest dressmaker’s invention, and magically rejuvenated, as though again and amiin dipped in the fountain of perpetual youth —bore conclusive testimony, she bravely reinforced her fund from time to time.'
 'The late Mr. A. Bratley, my grandfather, was indeed one of those rude, laborious, and serviceable persons whose office is to make money — or perhaps I should say to accumulate the means of enjoyment —for the upper classes of society. But my father, the late Mr. Harold Chylde, had gentlemanly tastes. llow can I blame him ? I hav

In [93]:
#load other dependencies 
import argparse
import json
import os

import numpy as np
import openai
import pandas as pd
import random
import time
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import os

In [94]:
title='atlantic'
subtitle='test'
stim_set=title+'-'+subtitle
seed=1
temperature=0.1 #want a low baking temp to have little variability or creativity 
engine='gpt-3.5-turbo' #change this to use different models
n_context=1
cache = True   
resume=False
# MIDI='freq' #or 'name', 'number', 'freq'
audience='People'
item='Text'

load_dotenv()

apiKey = os.environ.get('steven_key') #for stevens key 

# apiKey = os.environ.get('angela_key') #for angelas key 


openai.api_key = apiKey

if cache:
    random.seed(seed)

In [51]:
def generate_prompt(current):#(example_idxs, current_pair):
    prompt = f"""Your task is to classify a piece of as either formal or informal. 
You will be provided with two examples of formal text and two examples of informal text. Answer only with a number: 1 if formal, and 0 if informal.  

Text: {examples.Example.iloc[0]}
Rating: {examples.Label.iloc[0]}
    
Text: {examples.Example.iloc[1]}
Rating: {examples.Label.iloc[1]}

Text: {examples.Example.iloc[2]}
Rating: {examples.Label.iloc[2]}

Text: {examples.Example.iloc[3]}
Rating: {examples.Label.iloc[3]}
    
Text: {inputs[current]}
Rating:
"""
    if n_context==0:
        prompt = f"""Is the following piece of text formal or informal? Answer only with a number: 1 if formal, and 0 if informal Here is the text:\n{inputs[current]}"""
    
    return prompt

cache_folder = f'cache/{stim_set}'
os.makedirs(cache_folder, exist_ok=True)
def create_cache_filename():
    filename = f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}'
    # if args.shuffle_context_each_draw:
    #     filename += '-shuffle'
    return os.path.join(cache_folder, filename + '.json')

if not resume:
    visited = []
    predicted_ratings = np.zeros((len(inputs)))
    request_count = 0

cache = {}

if cache and os.path.exists(create_cache_filename()):
    cache = json.load(open(create_cache_filename()))
cached_keys = list(cache.keys())

for idx1, bname1 in enumerate(tqdm(inputs)):
    current = idx1
    if current in visited and predicted_ratings[current]!=0:
        print(f'Already visited {current}')
        continue

    visited.append(current)

    key = f'{current}'
    if key in cached_keys:
        choices = cache[key]['choices']
        print('Using cached choices for key', key)
    else:
        prompt = generate_prompt(current)#(example_idxs, current_pair)
        response=False
        i=0
        while not response:
            i+=1
            try:
                response = openai.ChatCompletion.create(
                    model=engine,
                    messages=[{'role':'user', 'content':prompt}],
                    temperature=temperature,
                    timeout=10
                )
            except Exception as e:
                print(f'Attempt {i} failed')
                time.sleep(5)
        choices = [dict(choice.items()) for choice in response.choices]

        cache[key] = {
            'prompt': prompt,
            'choices': choices,
            'created': response.created
        }

        request_count += 1
        if cache and request_count % 5 == 0:
            json.dump(cache, open(create_cache_filename(), 'w'))

    try:
        answer = choices[0]['message']['content'].replace('\n', '').strip()
        predicted_ratings[idx1] = int(answer)
    except:
        print('Error', cache[key])

os.makedirs(f'predictions/{stim_set}', exist_ok=True)
np.save(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', predicted_ratings)


  0%|          | 0/50 [00:00<?, ?it/s]

In [63]:
df=pd.read_csv('/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/master_data.csv')
# df_gpt4 = np.zeros((predicted_ratings.size, 3))
# df_gpt4[np.arange(predicted_ratings.size), predicted_ratings.astype(int)] = 1
predicted_ratings=np.load(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', allow_pickle=True)
df.head()

,file_name,year,Text,formality_rating,gpt_coding_final,angela_formality_rating,lyle_formality_rating,ben_formality_rating,steven_formality_rating,interrater_reliability,overall_inter_rater_reliability
0,627327-miss-wimples-hoop.html,1858,"But Miss Wimple’s proud and honorable fund, fo...",NaN,fiction,1,1,1,1,1.000000,0.280369
1,628188-the-question-of-the-hour.txt,1861,"However true this may be in the main, — and it...",NaN,non-fiction,1,1,1,1,1.000000,0.280369
2,628073-saccharissa-mellasys.html,1861,"The late Mr. A. Bratley, my grandfather, was i...",NaN,fiction,1,1,1,1,1.000000,0.280369
3,628531-notes-of-a-pianist-iii.html,1865,Unquestionably there are many blanks in certai...,NaN,non-fiction,1,0,1,1,-0.222222,0.280369
4,536646-the-pressure-upon-congress.html,1870,This is really a matter of first-rate importan...,NaN,non-fiction,1,0,1,1,-0.222222,0.280369


In [64]:
predicted_ratings # just to check it ran smoothly
df.columns

Index(['file_name', 'year', 'Text', 'formality_rating', 'gpt_coding_final',
       'angela_formality_rating', 'lyle_formality_rating',
       'ben_formality_rating', 'steven_formality_rating',
       'interrater_reliability', 'overall_inter_rater_reliability'],
      dtype='object')

In [68]:
df_gpt35=pd.DataFrame(predicted_ratings, columns=['formality_ratings'])
df_gpt35['Text']=df.Text.values
df_gpt35['file_name']=df.file_name.values


In [69]:
#save the work
output_directory = '/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/'
os.makedirs(output_directory, exist_ok=True)

df_gpt35.to_csv(os.path.join(output_directory, f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}.csv'), index=False)